In [1]:
import fasttext
model = fasttext.load_model('./util/cc.en.300.bin')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import json

from tqdm import tqdm

C:\Users\Baha Tegar\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import util.normalization as norm
import util.model as models
import util.utility as util

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


ValueError: Error loading wordnet: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:
pd.set_option("display.max_rows", 250)

In [ ]:
df = pd.read_csv("./mcd_result/base_mcd.csv")

df['aspect'] = df['aspect'].apply(lambda x: x.split(", "))
df['token_sentence'] = df['token_sentence'].apply(lambda x: x.split('.\n'))
# df['topic_keys'] = df['topic_keys'].apply(lambda x: x.split(", "))
print(df.info())
df.head()

In [ ]:
df_an = df[['reviewer_id', 'aspect']].copy()
df_an.head()

# Coding Process

In [ ]:
# map_categorize = {
#     0: 'order and food',
#     1: 'service and time',
#     2: 'order and food',
#     3: 'order and food',
#     4: 'service and time',
#     5: 'order and food',
#     6: 'service and time',
# }

# df['category'] = df['topic'].apply(lambda x: map_categorize[x])

# print(df.info())
# df.head()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text = ""
aspects = df_an['aspect'].values
for i in range(df_an.shape[0]):
    text = text + " " + " ".join(list(aspects[i]))
    

text = text.strip()

wordcloud = WordCloud(background_color='white').generate(text)
plt.style.use('classic')
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
words_count = {}

words = text.split()

for word in words:
    word = word.lower()
    if word in words_count:
        words_count[word] += 1
    else:
        words_count[word] = 1
        
# words_count = dict(
#                 sorted(words_count.items(),
#                        key=lambda x: x[1],
#                        reverse=True))

In [ ]:
df_word = pd.DataFrame(data=words_count.values(),
                       index=words_count.keys(), 
                       columns=['count'])

df_word.head(25).sort_values(by='count', ascending=True).plot(kind='barh')
plt.show()

In [ ]:
# Show 75% words

threshold = df_word.quantile(q=0.75).iloc[0]
# threshold = 0.25 * df_word.max().iloc[0]
# threshold = df_word.mean().iloc[0]
print(threshold)

df_plot = df_word[df_word['count'] > threshold]\
        .head(50)

ax = df_plot\
        .sort_values(by='count', ascending=True)\
        .plot(kind='barh', figsize=(10, 20))

for container in ax.containers:
    ax.bar_label(container)

plt.show()

## A. First Cycle

In [ ]:
list_first_cycle = list(df_word[df_word['count'] > threshold].index)
print(len(list_first_cycle))

# Additional filter
list_first_cycle = [word for word in list_first_cycle if len(word) >= 3]

def first_cycle(x):
    global list_first_cycle
        
    set1 = set(" ".join(x).split(" "))
    set2 = set(list_first_cycle)
    
    temp = set1 & set2
    
    if len(temp) != len(set1):
        temp.add("other")
    return temp

def get_other(x):
    global list_first_cycle
    
    set1 = set(" ".join(x).split(" "))
    set2 = set(list_first_cycle)
    
    temp = set1.difference(set2)
    
    return temp

def get_residu(x):
    texts = x['residu']
    array = x['aspect']

    temp = []
    print(texts)
    for text in texts:
        pattern = re.compile(rf"{text}")
        
        result = [element for element in array if pattern.search(element)]
        temp = temp + result

    return set(temp)



tqdm.pandas()
df_an['first_cycle'] = df_an['aspect'].progress_apply(lambda x: first_cycle(x))

df_an['residu'] = df_an['aspect'].progress_apply(lambda x: get_other(x))

df_an['residu'] = [get_residu(x[-1]) for x in df_an.iterrows()]

df_an = df_an.explode('first_cycle').reset_index(drop=True)
df_an

## B. Second Cycle

In [ ]:
map_second_code = {'situation': ['normal', 'line', 'large', 'long', 'regular', 'people', 'wait',
                                 'night', 'hour', 'wrong', 'minute', 'mess', 'day'], 
                   'place': ['mcdonald', 'mcdonalds', 'location', 'place', 'car', 'restaurant', 'window', 'store', 'drive',],
                   'emotion': ['great', 'best', 'bad', 'great', 'horrible', 'good', 'happy', 'rude',], 
                   'service': ['staff', 'customer', 'service', 'location', 'regular', 'time', 'lady', 'experience', 'crew',
                               'manager', 'employee', 'price', 'management', 'attitude', 'work', 'issue'],
                   'food': ['food', 'order', 'chicken', 'sandwich', 'meal', 'fry',
                            'nugget', 'item', 'cream', 'pickle', 'strawberry', 'burger'],
                   'drink': ['coffee', 'cup', 'item', 'drink', 'cream', 'strawberry', 'ice'] 
                  }

def second_cycle(x):
    temp = set()
    for key in map_second_code.keys():
        if x in map_second_code[key]:
            temp.add(key)
#     if len(temp) == 0:
#         temp.add('other')
        
    return temp


tqdm.pandas()
df_an['second_cycle'] = df_an['first_cycle'].progress_apply(lambda x: second_cycle(x))
# # EXPERIMENT
experiment = df_an[['reviewer_id', 'aspect', 'residu', 'first_cycle', 'second_cycle']].copy()


In [ ]:
# Extract residu

def extract_residu(x):
    global map_second_code
    
    residu = x['residu']
    first_cycle = x['first_cycle']
    second_keys = list(map_second_code.keys())
    temp = []
    
    if first_cycle != 'other':
        return set()
    
    for s in residu:
        result = util.get_nearest_word(s, second_keys,
                                       model, threshold=.35)
        
        if result:
            temp.append(result)
    return set(temp)

def merge_residu(x):
    set1 = x['second_cycle']
    set2 = x['extract_residu']
    
    result = set1.union(set2)
    
    if len(result) == 0:
        return set(['other'])
    return set1.union(set2)


df_an['extract_residu'] = [extract_residu(x[-1]) for x in df_an.iterrows()]
df_an['second_cycle'] = [merge_residu(x[-1]) for x in df_an.iterrows()]


df_an = df_an.explode('second_cycle').reset_index(drop=True).drop('extract_residu', axis=1)

df_an

In [ ]:
# Store residu data

residu_frame = df_an.groupby('reviewer_id')['residu']\
                .apply(lambda x: set().union(*x))\
                .reset_index()

residu_frame['residu'] = residu_frame['residu'].apply(lambda x: ", ".join(list(x)))

In [ ]:
result = df_an[['reviewer_id', 'first_cycle', 'second_cycle']]\
            .copy()\
            .reset_index()\
            .rename(columns={'index': 'node_id'})

print(result.info())
result.head()

## C. Categorizing

In [ ]:
map_categorizing = {
    'sentiment': ['emotion'],
    'place and service': ['service', 'situation', 'place'],
    'food and drink': ['food', 'drink'],
}

def final_categorizing(x):
    for key in map_categorizing.keys():
        if x in map_categorizing[key]:
            return key
    return "other"

tqdm.pandas()
result['category'] = result['second_cycle'].progress_apply(lambda x: final_categorizing(x))

In [ ]:
result

# Linking Nodes

In [ ]:
import networkx as nx

for cat in np.unique(df_an['second_cycle'].values):
    df_plot = df_an[df_an['second_cycle'] == cat]
    G = nx.from_pandas_edgelist(df_plot, 'first_cycle', 'second_cycle')
    nx.draw_kamada_kawai(G, with_labels=True, node_color='skyblue', node_size=1000)
    plt.show()

# Add Residu to Data Base (Update Base)

In [ ]:
df = df.merge(residu_frame, on='reviewer_id', how='left')

df['aspect'] = df['aspect'].apply(lambda x: ", ".join(list(x)))
df['token_sentence'] = df['token_sentence'].apply(lambda x: ", ".join(list(x)))

print(df.info())
df.head()

# Save Important Files

In [ ]:
result.to_csv("./mcd_result/nodes_mcd.csv", index=False)

In [ ]:
df.to_csv("./mcd_result/base_mcd_v1.csv", index=False)

In [ ]:
with open("./mcd_result/map_second_mcd.json", 'w') as file:
    json.dump(map_second_code, file)
    
with open("./mcd_result/map_category_mcd.json", "w") as file:
    json.dump(map_categorizing, file)
    
with open("./mcd_result/list_first_cycle_mcd.json", "w") as file:
    json.dump(list_first_cycle, file)